# RAG Based on parent document
- Hybrid Search
- ReRanker
- [Parent Document](https://medium.aiplanet.com/advanced-rag-providing-broader-context-to-llms-using-parentdocumentretriever-cc627762305a)
    

## Setting
 - Auto Reload
 - path for utils

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
module_path = "../../../.."
sys.path.append(os.path.abspath(module_path))

## 1. Create a Bedrock Client

In [3]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

### ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----
- os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
- os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
- os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
- os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."

In [6]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

aws_region = os.environ.get("AWS_DEFAULT_REGION", None)
print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models(verbose=False))

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Claude-V3-5-Sonnet': 'anthropic.claude-3-5-sonnet-20240620-v1:0',
 'Claude-V3-Haiku': 'anthropic.claude-3-haiku-20240307-v1:0',
 'Claude-V3-Opus': 'anthropic.claude-3-sonnet-20240229-v1:0',
 'Claude-V3-Sonnet': 'anthropic.claude-3-sonnet-20240229-v1:0',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'T

## 2. Load Titan Embedding and LLM Claude model

### Load LLM

In [7]:
from langchain_aws import ChatBedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [8]:
llm_text = ChatBedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V3-5-Sonnet"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens": 1024,
        "stop_sequences": ["\n\nHuman"],
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)
llm_text

ChatBedrock(callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7feb580d3700>], client=<botocore.client.BedrockRuntime object at 0x7feb31e749d0>, model_id='anthropic.claude-3-sonnet-20240229-v1:0', model_kwargs={'max_tokens': 1024, 'stop_sequences': ['\n\nHuman']}, streaming=True)

### Select Embedding model

In [9]:
from utils.rag import KoSimCSERobertaContentHandler, SagemakerEndpointEmbeddingsJumpStart

In [32]:
def get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name=None):
    
    if is_bedrock_embeddings:
        # We will be using the Titan Embeddings Model to generate our Embeddings.
        from langchain.embeddings import BedrockEmbeddings
        llm_emb = BedrockEmbeddings(
            client=boto3_bedrock,
            model_id=bedrock_info.get_model_id(
                model_name="Titan-Text-Embeddings-V2"
            )
        )
        print("Bedrock Embeddings Model Loaded")

    elif is_KoSimCSERobert:
        LLMEmbHandler = KoSimCSERobertaContentHandler()
        endpoint_name_emb = endpont_name
        llm_emb = SagemakerEndpointEmbeddingsJumpStart(
            endpoint_name=endpoint_name_emb,
            region_name=aws_region,
            content_handler=LLMEmbHandler,
        )
        print("KoSimCSERobert Embeddings Model Loaded")
    else:
        llm_emb = None
        print("No Embedding Model Selected")

    return llm_emb

#### [Important] If is_KoSimCSERobert == True, please make sure to include endpoint_name.

In [33]:
is_bedrock_embeddings = True
is_KoSimCSERobert = False
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_KoSimCSERobert
##############################
if is_KoSimCSERobert: endpont_name = "<endpoint-name>"
else: endpont_name = None
##############################

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name)   

Bedrock Embeddings Model Loaded


## 3. Depoly ReRanker model (if needed)

In [34]:
import json
import sagemaker
from sagemaker.huggingface import HuggingFaceModel

In [13]:
# depoly = False
depoly = True

In [14]:
if depoly:

    try:
        role = sagemaker.get_execution_role()
    except ValueError:
        iam = boto3.client('iam')
        role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

    # Hub Model configuration. https://huggingface.co/models
    hub = {
        'HF_MODEL_ID':'BAAI/bge-reranker-large',
        'HF_TASK':'text-classification'
    }

    # create Hugging Face Model Class
    huggingface_model = HuggingFaceModel(
        transformers_version='4.26.0',
        pytorch_version='1.13.1',
        py_version='py39',
        env=hub,
        role=role, 
    )

    # deploy model to SageMaker Inference
    predictor = huggingface_model.deploy(
        initial_instance_count=1, # number of instances
        instance_type='ml.g5.xlarge' # instance type
    )

    print(f'Accept: {predictor.accept}')
    print(f'ContentType: {predictor.content_type}')
    print(f'Endpoint: {predictor.endpoint}')

---------!

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Accept: ('application/json',)
ContentType: application/json
Endpoint: huggingface-pytorch-inference-2024-10-06-04-43-38-392


#### Save reranker endpoint to Parameter Store

In [15]:
if depoly:

    import boto3
    from utils.ssm import parameter_store

    region=boto3.Session().region_name
    pm = parameter_store(region)

    pm.put_params(
        key="reranker_endpoint",
        value=f'{predictor.endpoint}',
        overwrite=True,
        enc=False
    )

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Parameter stored successfully.


## 4. Invocation (prediction)

In [35]:
from utils.ssm import parameter_store

In [36]:
region=boto3.Session().region_name
pm = parameter_store(region)

In [37]:
runtime_client = boto3.Session().client('sagemaker-runtime')
print (f'runtime_client: {runtime_client}')

runtime_client: <botocore.client.SageMakerRuntime object at 0x7feab2dda290>


In [38]:
endpoint_name = pm.get_params(
    key="reranker_endpoint",
    enc=False
)
deserializer = "application/json"

In [39]:
payload = json.dumps(
    {
        "inputs": [
            {"text": "I hate you", "text_pair": "I don't like you"},
            {"text": "He hates you", "text_pair": "He like you"}
        ]
    }
)

In [40]:
%%time
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Accept=deserializer,
    Body=payload
)
## deserialization
out = json.loads(response['Body'].read().decode()) ## for json
print (f'Response: {out}')

Response: [{'label': 'LABEL_0', 'score': 0.9987371563911438}, {'label': 'LABEL_0', 'score': 0.0028581616934388876}]
CPU times: user 14.7 ms, sys: 0 ns, total: 14.7 ms
Wall time: 94 ms


## 5. Define LangChain OpenSearch VectorStore
### Prerequisites
- The OpenSearch Index should have been created through 01_preprocess_docs/02_load_docs_opensearch.ipynb.
#### [Important] The following authentication information should be entered in the AWS Parameter Store first:
- Refer to 01_preprocess_docs/01_parameter_store_example.ipynb

In [41]:
opensearch_domain_endpoint = pm.get_params(
    key="opensearch_domain_endpoint",
    enc=False
)

opensearch_user_id = pm.get_params(
    key="opensearch_user_id",
    enc=False
)

opensearch_user_password = pm.get_params(
    key="opensearch_user_password",
    enc=True
)

In [42]:
opensearch_domain_endpoint = opensearch_domain_endpoint
rag_user_name = opensearch_user_id
rag_user_password = opensearch_user_password

http_auth = (rag_user_name, rag_user_password) # Master username, Master password

### Set Index Name
- Enter the OpenSearch Index name created through the previous notebook 01_preprocess_docs/02_load_docs_opensearch.ipynb
- Use the index created for parent documents

In [43]:
index_name = opensearch_user_password = pm.get_params(
    key="opensearch_index_name",
    enc=True
)

print (f'index_name: {index_name}')

index_name: v01-genai-poc-parent-doc-retriever


### Create OpenSearch Client

In [44]:
from utils.opensearch import opensearch_utils

In [45]:
os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

## 6. Retriever based on Hybrid Search + ParentDocument + ReRanker
- You can define a **Custom Retriever** by inheriting the **BaseRetriever** class provided by LangChain.

### Hybrid Search
Hybrid search in OpenSearch works in the following way:

- (1) Calculate relevant score for each document through semantic search
- (2) Calculate relevant score for each document through lexical search
- (3-1) If the rank-fusion method is "simple weighted":
    - Perform normalization on the calculated scores
    - The highest score in the overall results becomes 1.0 through the normalization process
- (3-2) If the rank-fusion method is "Reciprocal Rank Fusion (RRF)":
    - Paper: https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf
    - Desc: https://medium.com/@sowmiyajaganathan/hybrid-search-with-re-ranking-ff120c8a426d
    - **RRF uses ranking information instead of scores, so score normalization is not necessary**
    - - ![rrf.png](../../../../10_advanced_question_answering/img/rrf.png)

LangChain provides an API for RRF under the name "Ensemble Retriever":

- https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble

### Parent Document
- [Parent Document](https://towardsdatascience.com/forget-rag-the-future-is-rag-fusion-1147298d8ad1)
![parent-document.png](../../../../imgs/parent-document.png)

## Parent documents in Hybrid search
- Lexical search: parent documents only
- Semantic search: child documents first, getting parent documents corresponding to that child document

In [46]:
from utils.rag import OpenSearchHybridSearchRetriever

- Example of filter settings
- filter=[ <BR>
    　{"term": {"metadata.[**your_metadata_attribute_name**]": "**your first keyword**"}}, <BR>
    　{"term": {"metadata.[**your_metadata_attribute_name**]": "**your second keyword**"}},<BR>
]

In [47]:
opensearch_hybrid_retriever = OpenSearchHybridSearchRetriever(
    os_client=os_client,
    index_name=index_name,
    llm_text=llm_text, # llm for query augmentation in both rag_fusion and HyDE
    llm_emb=llm_emb, # Used in semantic search based on opensearch 

    # option for lexical
    minimum_should_match=0,
    filter=[],

    # option for search
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"], rank fusion 방식 정의
    ensemble_weights=[.51, .49], # [for semantic, for lexical], Semantic, Lexical search 결과에 대한 최종 반영 비율 정의
    reranker=True, # enable reranker with reranker model
    reranker_endpoint_name=endpoint_name, # endpoint name for reranking model
    parent_document = True, # enable parent document

    # option for async search
    async_mode=True,

    # option for output
    k=5, # 최종 Document 수 정의
    verbose=True,
)

### Retrieval example
- default search

In [48]:
from utils.rag import show_context_used

In [49]:
#query = "vidio max size?"
query = "What is knox?"

In [50]:
%%time
search_hybrid_result = opensearch_hybrid_retriever.get_relevant_documents(query)

print("\n==========  Results  ==========\n")
print(f'1. question: {query}')
print (f'2. # documents: {len(search_hybrid_result)}')
print("3. Documents: \n")

show_context_used(search_hybrid_result)

===== ParentDocument =====
filter: [{'bool': {'should': [{'term': {'metadata.family_tree': 'child'}}, {'term': {'metadata.family_tree': 'parent_table'}}, {'term': {'metadata.family_tree': 'parent_image'}}]}}]
# child_docs: 5
# parent docs: 5
# duplicates: 0
##############################
async_mode
##############################
True
##############################
reranker
##############################
True
##############################
rag_fusion
##############################
False
##############################
HyDE
##############################
False
##############################
parent_document
##############################
True
##############################
complex_document
##############################
False
##############################
similar_docs_semantic
##############################

Score: 1.0
['What is Knox Suite?. Knox Suite is a bundled offering designed to help enterprise IT admins better manage your fleet of devices. It includes individual services such as K

- update parameters
    - **parent_document=False**

In [51]:
opensearch_hybrid_retriever.update_search_params(
    k=5,
    minimum_should_match=0,
    #filter=[],
    filter=[
        #{'term': {'metadata.project': 'KS'}},
        #{'term': {'metadata.family_tree': 'child'}},
    ],
    reranker=True,
    reranker_endpoint_name=endpoint_name,
    parent_document=False, # enable parent document
    verbose=True,
)

In [52]:
query = "What is knox?"
search_hybrid_result = opensearch_hybrid_retriever.get_relevant_documents(query)

print("\n==========  Results  ==========\n")
print(f'1. question: {query}')
print(f'2. # documents: {len(search_hybrid_result)}')
print("3. Documents: \n")

show_context_used(search_hybrid_result)

##############################
async_mode
##############################
True
##############################
reranker
##############################
True
##############################
rag_fusion
##############################
False
##############################
HyDE
##############################
False
##############################
parent_document
##############################
False
##############################
complex_document
##############################
False
##############################
similar_docs_semantic
##############################

Score: 1.0
['What is Knox Suite?. Knox Suite is a bundled offering designed to help enterprise IT admins better manage your fleet of devices. It includes individual services such as Knox Platform for Enterprise, Knox Mobile Enrollment, Knox Manage, Knox E-FOTA, and Knox Asset Intelligence. Key features of Knox Suite include: Secure - Ensure your business data is protected with managed security features at your control. Deploy - Enroll c

## 5. RAG using RetrievalQA powered by LangChain

In [53]:
from utils.rag import prompt_repo
from utils.rag import run_RetrievalQA
from langchain.prompts import PromptTemplate

### Prompting
- [TIP] For the instruction in the prompt, you may get better results by using English rather than Korean.

In [54]:
from textwrap import dedent

system_prompt = dedent(
    """
    You are a master answer bot designed to answer user's questions.
    I'm going to give you contexts which consist of texts, tables and images.
    Read the contexts carefully, because I'm going to ask you a question about it.
    """
)

human_prompt = dedent(
    """
    Here is the contexts as texts: <contexts>{contexts}</contexts>

    First, find a few paragraphs or sentences from the contexts that are most relevant to answering the question.
    Then, answer the question as much as you can.

    Skip the preamble and go straight into the answer.
    Don't insert any XML tag such as <contexts> and </contexts> when answering.
    Answer in english.

    Here is the question: <question>{question}</question>

    If the question cannot be answered by the contexts, say "No relevant contexts".
    """
)

('\n'
 '                        You are a master answer bot designed to answer '
 "user's questions.\n"
 "                        I'm going to give you contexts which consist of "
 'texts, tables and images.\n'
 "                        Read the contexts carefully, because I'm going to "
 'ask you a question about it.\n'
 '                        ')


### Update Search Params (Optional)

In [55]:
from utils.rag import rag_chain
from langchain.schema.output_parser import StrOutputParser

In [56]:
opensearch_hybrid_retriever.update_search_params(
    k=6,
    minimum_should_match=0,
    # filter=[
    #     {'term': {'metadata.family_tree': 'child'}},
    # ],
    ensemble_weights=[0.51, 0.49], #semantic, lexical

    reranker=True,
    reranker_endpoint_name=endpoint_name,

    parent_document=True, # enable parent document
    verbose=False
)

### Request

In [57]:
from utils.rag import rag_chain
qa = rag_chain(
    llm_text=llm_text,
    retriever=opensearch_hybrid_retriever,
    system_prompt=system_prompt,
    human_prompt=human_prompt,
    return_context=True,
    verbose=False,
    #multi_turn=True
)

In [58]:
#query = "vidio max size?"
query = "How does Knox E-FOTA enable enterprises to deploy OS updates remotely without requiring user interaction?"
query = "What feature of Knox E-FOTA allows IT administrators to ensure that devices have the same OS version installed across the entire fleet of devices?"
query = "How does Knox E-FOTA allow IT admins to securely push updates to enterprise mobile devices while maintaining compatibility with in-house apps?"
query = "What are the three editions of Knox E-FOTA service that have different features?"
query = "How do you obtain the Knox E-FOTA client APK that is needed to install the app through an EMM?"

response, contexts = qa.invoke(
    query = query
)

#show_context_used(contexts)

Knox E-FOTA 클라이언트 APK를 얻는 방법은 다음과 같습니다:

1. Knox 관리 포털에 로그인합니다.
2. 왼쪽 사이드바에서 'Knox E-FOTA'를 클릭합니다. 
3. 'Devices > All Devices'로 이동합니다.
4. 'Direct Download'를 클릭합니다.
5. 'Direct Download' 팝업에서 APK 파일의 다운로드 링크를 복사합니다.

이렇게 하면 Knox E-FOTA 클라이언트 APK 파일이 로컬 컴퓨터에 다운로드됩니다.

In [59]:
print("##################################")
print("query: ", query)
print("##################################")

print (colored("\n\n### Answer ###", "blue"))
print_ww(response)

print (colored("\n\n### Contexts ###", "green"))
show_context_used(contexts)

##################################
query:  How do you obtain the Knox E-FOTA client APK that is needed to install the app through an EMM?
##################################


### Answer ###
<generator object RunnableSequence.stream at 0x7feab00e2f10>


### Contexts ###

-----------------------------------------------
1. Chunk: 1517 Characters
-----------------------------------------------
Install an app through an EMM. Explains how to install an app through an EMM. This section provides
general instructions on how to install an app (like the Knox E-FOTA app) using various EMMs. The
general workflow is as follows: 1. Obtain the Knox E-FOTA client APK. 2. In your EMM's console, add
the Knox E-FOTA client as an internal app. 3. Set a policy to install the app to your target
devices. If your EMM has the capability to automatically launch the client, we strongly recommend
you use it. Enrollment cannot be completed until the client is launched on the device. Obtain the
Knox E-FOTA client AP